In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

In [162]:
df = scale(pd.read_csv('Dynamicgraph_ACCESS7.csv').iloc[:,1:-1].values)

In [163]:
df.shape

(200, 16)

In [164]:
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if df[i][j] == 0:
            df[i][j] = 1

In [165]:
data = np.abs(df.reshape(df.shape[0],4,4))

In [166]:
def preprocess_data(data):
    # 替换NaN值为均值
    nan_mask = np.isnan(data)
    data[nan_mask] = np.nanmean(data)
    
    # 或者替换为特定的值，例如 0
    # data[nan_mask] = 0  # 替换为0
    
    return data

In [167]:
data = preprocess_data(data)

In [168]:
data[5]

array([[1.        , 0.01805094, 0.56593093, 0.28111563],
       [0.01805094, 1.        , 0.29698762, 0.24295839],
       [0.56593093, 0.29698762, 1.        , 0.27893633],
       [0.28111563, 0.24295839, 0.27893633, 1.        ]])

In [169]:
def entropy(X):
    E = []
    for i in range(X.shape[0]):
        P = []
        for j in range(X.shape[1]):
            if i !=j:
                e = -X[i][j]*np.log(X[i][j])
                P.append(e)
        P = np.array(P)
        E.append(np.sum(P))
    return np.array(E)

In [170]:
def graphentropy(X):
    E = []
    for i in range(X.shape[0]):
        e = entropy(X[i])
        E.append(np.sum(e))
    return np.array(E)

In [171]:
entropies = graphentropy(data)

In [172]:
np.inf in entropies

False

In [173]:
distances = cdist(entropies.reshape(-1, 1), entropies.reshape(-1, 1), metric='euclidean')
distances.shape


(200, 200)

In [174]:
most_similar_indices = np.argmin(distances, axis=1)


In [175]:
most_similar_indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [176]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=16):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

# Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, ff_dim, num_layers):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x

# Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, bottleneck_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [177]:
input_dim = 16
embed_dim = 16
num_heads = 4
ff_dim = 128
num_layers = 3
hidden_dim = 32
bottleneck_dim = 16
lr = 0.01
num_epochs = 50

In [178]:
transformer_encoder = TransformerEncoder(input_dim, embed_dim, num_heads, ff_dim, num_layers)
autoencoder = Autoencoder(input_dim, hidden_dim, bottleneck_dim)

In [179]:
optimizer = optim.Adam(list(transformer_encoder.parameters()) + list(autoencoder.parameters()), lr=lr)


In [180]:
mse_loss = nn.MSELoss()


In [181]:
len(data)

200

In [182]:
data[0]

array([[1.        , 0.78036223, 0.98857452, 0.84882841],
       [0.78036223, 1.        , 1.16690413, 0.42749557],
       [0.98857452, 1.16690413, 1.        , 1.14361819],
       [0.84882841, 0.42749557, 1.14361819, 1.        ]])

In [183]:
x = torch.tensor(data[2].flatten(), dtype=torch.float32).unsqueeze(0)
similar_x = torch.tensor(data[most_similar_indices[2]].flatten(), dtype=torch.float32).unsqueeze(0)

similar_x

tensor([[1.0000e+00, 9.9087e-01, 5.0363e-01, 2.8071e-01, 9.9087e-01, 1.0000e+00,
         1.2818e+00, 1.2847e+00, 5.0363e-01, 1.2818e+00, 1.0000e+00, 5.3874e-01,
         2.8071e-01, 1.2847e+00, 5.3874e-01, 2.2204e-16]])

In [184]:
data[most_similar_indices[2]],data[5],most_similar_indices[2]

(array([[1.00000000e+00, 9.90870323e-01, 5.03633203e-01, 2.80707435e-01],
        [9.90870323e-01, 1.00000000e+00, 1.28182437e+00, 1.28466234e+00],
        [5.03633203e-01, 1.28182437e+00, 1.00000000e+00, 5.38744045e-01],
        [2.80707435e-01, 1.28466234e+00, 5.38744045e-01, 2.22044605e-16]]),
 array([[1.        , 0.01805094, 0.56593093, 0.28111563],
        [0.01805094, 1.        , 0.29698762, 0.24295839],
        [0.56593093, 0.29698762, 1.        , 0.27893633],
        [0.28111563, 0.24295839, 0.27893633, 1.        ]]),
 2)

In [185]:
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(data)):
        x = torch.tensor(data[i].flatten(), dtype=torch.float32).unsqueeze(0)
        similar_x = torch.tensor(data[most_similar_indices[i]].flatten(), dtype=torch.float32).unsqueeze(0)

        # Transformer Encoder
        transformer_output = transformer_encoder(x)

        # Autoencoder
        encoded, decoded = autoencoder(similar_x)

        # 损失计算
        loss1 = mse_loss(transformer_output, encoded)
        loss2 = mse_loss(similar_x, decoded)
        loss = loss1 + loss2

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(data)}')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 16])) that is different to the input size (torch.Size([1, 1, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.3545854719541967
Epoch 2/50, Loss: 0.27047521630302074
Epoch 3/50, Loss: 0.24667332739569248
Epoch 4/50, Loss: 0.240764530133456
Epoch 5/50, Loss: 0.23957975958473982
Epoch 6/50, Loss: 0.23454898681491612
Epoch 7/50, Loss: 0.2352264958806336
Epoch 8/50, Loss: 0.23373521105386316
Epoch 9/50, Loss: 0.23426728431135416
Epoch 10/50, Loss: 0.23373165619559585
Epoch 11/50, Loss: 0.23311674419790507
Epoch 12/50, Loss: 0.23301371506415308
Epoch 13/50, Loss: 0.23244138691574334
Epoch 14/50, Loss: 0.23129835310392083
Epoch 15/50, Loss: 0.23002781852148474
Epoch 16/50, Loss: 0.2294008821155876
Epoch 17/50, Loss: 0.23002006785012782
Epoch 18/50, Loss: 0.22870499855838716
Epoch 19/50, Loss: 0.2315006744582206
Epoch 20/50, Loss: 0.2294099312182516
Epoch 21/50, Loss: 0.2304657519236207
Epoch 22/50, Loss: 0.2294485124014318
Epoch 23/50, Loss: 0.22889952807687222
Epoch 24/50, Loss: 0.23051330741494894
Epoch 25/50, Loss: 0.23016918311826884
Epoch 26/50, Loss: 0.2323167350050062
Epoch

In [186]:
def get_embeddings(model, data):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for i in range(len(data)):
            x = torch.tensor(data[i].flatten(), dtype=torch.float32).unsqueeze(0)
            embedding = model(x)
            embeddings.append(embedding.squeeze(0).numpy())
    return np.array(embeddings)

# 获取嵌入向量
embeddings = get_embeddings(transformer_encoder, data)
#np.savetxt('Embedding_vector.csv', embeddings, delimiter=',')

In [187]:
np.save('Embeddings_ACCESS7_ETA.npy', embeddings)

print("Embeddings saved to embeddings.npy")

Embeddings saved to embeddings.npy
